In [2]:
#importa as bibliotecas necessarias
import findspark
findspark.init()

from os.path import isfile, join
import os, shutil
import pyspark
from os import listdir
import urllib
import pandas as pd

import unicodedata
import re

import random
from pyspark.sql import SparkSession, SQLContext

In [3]:
LANDED = 'C:/Users/970775/Documents/ProjectsPhyton/Bigdata/Projeto/landed/'
RAW = 'C:/Users/970775/Documents/ProjectsPhyton/Bigdata/Projeto/raw/'

In [4]:
#init spark context and spark session
builder = SparkSession.builder.appName("DOWNLOAD ARQUIVOS")
builder.config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")

spark = builder.getOrCreate()
sql = SQLContext(spark.sparkContext)

In [15]:
#Recupera a URL
print("Recupera informações de URL ...")
URL2017DETRAN = 'http://dados.recife.pe.gov.br/dataset/6399f689-f1a7-453b-b839-413bd665c355/resource/bb20e5ad-84ae-441c-84b0-4829b887a31a/download/relatorio-de-multas-implantadas-em-2017.csv'
URL2018DETRAN = 'http://dados.recife.pe.gov.br/dataset/6399f689-f1a7-453b-b839-413bd665c355/resource/0da61578-7f2d-42d6-9c61-ded3e0215437/download/relatorio-de-multas-implantadas-em-2018.csv'
URL2019DETRAN = 'http://dados.recife.pe.gov.br/dataset/6399f689-f1a7-453b-b839-413bd665c355/resource/5369f3fd-ce5f-4409-b9b1-19ff90c4f6ac/download/relatorio.csv'

print("Baixando dados de 2017 ....")
urllib.request.urlretrieve(URL2017DETRAN, "2017_INFRACOES.csv")
f = urllib.request.urlopen(URL2017DETRAN)
data = f.read()
#grava o arquivo 
with open("2017_INFRACOES.csv", "wb") as code:
    code.write(data)  

print("Baixando dados de 2018 ....")
urllib.request.urlretrieve(URL2018DETRAN, "2018_INFRACOES.csv")
f = urllib.request.urlopen(URL2018DETRAN)
data = f.read()
with open("2018_INFRACOES.csv", "wb") as code:
    code.write(data)     
    
print("Baixando dados de 2019 ....")
urllib.request.urlretrieve(URL2019DETRAN, "2019_INFRACOES.csv")
f = urllib.request.urlopen(URL2019DETRAN)
data = f.read()
with open("2019_INFRACOES.csv", "wb") as code:
    code.write(data)  
print("Finalizado ...")

Recupera informações de URL ...
Baixando dados de 2017 ....
Baixando dados de 2018 ....
Baixando dados de 2019 ....
Finalizado ...


In [5]:
# Diretorio

# Recupera lista de arquivos CSV em um diretorio
arquivos = [f for f in listdir(LANDED) if (isfile(join(LANDED, f)) and f.endswith('.csv')) ]

#Monta a estrutura
from pyspark.sql.types import  (StructType, 
                                StructField, 
                                DateType, 
                                BooleanType,
                                DoubleType,
                                IntegerType,
                                StringType,
                               TimestampType)
infracao_schema = StructType([StructField("datainfracao", DateType(), True),
                            StructField("horainfracao", DateType(), True),
                            StructField("infracao", StringType(), True ),
                            StructField("localcometimento", StringType(), True),
                            StructField("agenteequipamento", StringType(), True),
                            StructField("descricaoinfracao", StringType(), True  )
                            ])

In [24]:
# Para cada arquivo...
#selecionas as colunas, remove as barras e incluir "minus" , transforma em caixa baixa
csv = [pd.read_csv(f, sep=';', header=None, usecols = [0,1,3,4,5,7]).apply(lambda x: x.astype(str).str.lower()) for f in arquivos]
csv[0] = csv[0].replace('/','-')
#concatena
full_csv = pd.concat(csv, sort=False)
#salva
full_csv.to_csv( RAW + "infracao_anos.csv", index=False, encoding='utf-8-sig', sep=';', header=None)


In [25]:
full_csv

,0,1,3,4,5,7
0,datainfracao,horainfracao,agenteequipamento,infracao,descricaoinfracao,localcometimento
1,2017-01-01,20:12:24,código 8 - autos no talão eletrônico,5452,estacionar o veículo no passeio.,"avenida conselheiro aguiar, ao lado bompreco"
2,2017-01-01,07:38:21,código 3 - lombada eletrônica,7455,transitar em velocidade superior à máxima perm...,"av. marechal mascarenhas de moraes - semaforo,..."
3,2017-01-01,11:15:50,código 3 - lombada eletrônica,7463,transitar em velocidade superior à máxima perm...,"av. recife, semaforo 336"
4,2017-01-01,11:20:34,código 3 - lombada eletrônica,7455,transitar em velocidade superior à máxima perm...,"av. gov. agamenon magalhaes, semaforo 173."
5,2017-01-01,12:40:59,código 3 - lombada eletrônica,7463,transitar em velocidade superior à máxima perm...,"rua madre de deus, semaforo 020."
6,2017-01-01,13:03:06,código 3 - lombada eletrônica,7463,transitar em velocidade superior à máxima perm...,"av. recife, semaforo 336"
7,2017-01-01,14:25:32,código 3 - lombada eletrônica,7455,transitar em velocidade superior à máxima perm...,"av. marechal mascarenhas de moraes - semaforo,..."
8,2017-01-01,06:32:38,código 3 - lombada eletrônica,7455,transitar em velocidade superior à máxima perm...,"rua real da torre, semaforo 237."
9,2017-01-01,16:09:12,código 3 - lombada eletrônica,7455,transitar em velocidade superior à máxima perm...,"rua real da torre, semaforo 237."
